In [9]:
import numpy as np
import pandas as pd
import contractions
import yaml
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score 
from sklearn.preprocessing import LabelEncoder 
from catboost import Pool, CatBoostClassifier
from tqdm import tqdm
import re
import json
import nltk
from nltk import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aigozhiev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aigozhiev\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
with open('../build/labels.json', 'r') as f:
    mapa = json.load(f)
mapa

{'direct': {'Positive': 3, 'Neutral': 2, 'Negative': 1, 'Irrelevant': 0},
 'reverse': {'3': 'Positive',
  '2': 'Neutral',
  '1': 'Negative',
  '0': 'Irrelevant'}}

In [3]:
df = pd.read_csv('../data/twitter_training.csv')
df.columns = ['id','game','target','text']
df = df.drop('id', axis=1)
df["text"] = df['text'].apply(lambda x: str(x))
df["text"] = df.text.apply(lambda x: re.sub('[^A-Za-z0-9 ]+', '', x))
df['text'] = df.text.apply(lambda x: re.sub(r'[^\w\s]', '', x))
df["text"] = df['text'].str.lower()
df['text'] = df['text'].apply(lambda x: contractions.fix(x))
df['target'] = df['target'].apply(lambda x: mapa['direct'][x])

In [4]:
stopwords_nltk = nltk.corpus.stopwords
stop_words = stopwords_nltk.words('english')

In [5]:
y = df['target']
x = df.drop('target', axis=1)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [6]:
lbl = LabelEncoder()
X_train['game'] = lbl.fit_transform(X_train['game'])
X_test['game'] = lbl.transform(X_test['game'])

In [10]:
with open('../build/configs/cb_config.yaml', 'r') as f:
    params = yaml.safe_load(f)

params

{'model_config': {'loss_function': 'MultiClass',
  'random_seed': 42,
  'eval_metric': 'Accuracy',
  'use_best_model': True,
  'iterations': 1000,
  'learning_rate': 0.3}}

In [11]:
model = CatBoostClassifier(**params['model_config'])

train_p = Pool(X_train, 
               y_train, 
               cat_features=['game'], 
               text_features=['text'],
               feature_names=list(X_train))

valid_p = Pool(X_test,
               y_test, 
               cat_features=['game'], 
               text_features=['text'],
               feature_names=list(X_train))

hist = model.fit(train_p, eval_set=valid_p, verbose=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

KeyboardInterrupt: 

In [8]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.724844346254268

## If we add more iterations and tune hyperparameters we might get better results